In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pathlib import Path
import os
import fnmatch
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns
from descartes import PolygonPatch
import alphashape
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import Point
import math

In [ ]:
ProTcell_FolderLocation=r"C:\Users\data"  
dir=Path(ProTcell_FolderLocation)
os.chdir(ProTcell_FolderLocation)

# Tcell
for f in os.listdir(dir):
    if f.endswith("_data.txt"):
        dfT=pd.read_csv(f,sep="\t")
        if "Sample Name" in dfT.columns:
            dfT=dfT[["Path","Sample Name", "Phenotype", "Cell X Position NEW","Cell Y Position NEW"]]
            dfT=dfT.rename(columns={"Cell X Position NEW":"X",
                                   "Cell Y Position NEW":"Y"})
        elif "Sample.Name" in dfT.columns:
            dfT=dfT[["Path","Sample.Name", "Phenotype", "Cell.X.Position.NEW","Cell.Y.Position.NEW"]]
            dfT=dfT.rename(columns={"Sample.Name":"Sample Name",
                                   "Cell.X.Position.NEW":"X",
                                   "Cell.Y.Position.NEW":"Y"})
        dfT["Path"] = dfT["Sample Name"][0].split("x")[0].replace(" ","")
        #dfT["Path"] = dfT["Sample Name"][0].split("_")[0].replace(" ","")
        dfT = dfT.dropna(subset=["Phenotype"])
        cDC1WSI_FolderLocation=r"C:\Users\data\2"  
        dfT.reset_index(inplace=True)
        dir=Path(cDC1WSI_FolderLocation)
        for f in os.listdir(dir):
            if fnmatch.fnmatch(f, "*"+dfT["Path"][1]+"*"):
                dfcDC1WSI=pd.read_csv(cDC1WSI_FolderLocation+"\\"+f,sep="\t")
                if "Sample Name" in dfcDC1WSI.columns:
                    dfcDC1WSI=dfcDC1WSI[["Path","Sample Name", "Phenotype", "Cell X Position NEW","Cell Y Position NEW"]]
                    dfcDC1WSI=dfcDC1WSI.rename(columns={"Cell X Position NEW":"X",
                                           "Cell Y Position NEW":"Y"})
                elif "Sample.Name" in dfcDC1WSI.columns:
                    dfcDC1WSI=dfcDC1WSI[["Path","Sample.Name", "Phenotype", "Cell.X.Position","Cell.Y.Position"]]
                    dfcDC1WSI=dfcDC1WSI.rename(columns={"Sample.Name":"Sample Name",
                                                   "Cell.X.Position":"X",
                                                   "Cell.Y.Position":"Y"})
                dfcDC1WSI["Path"] = dfT["Sample Name"][1].split("x")[0].replace(" ","")
                dfcDC1WSI = dfcDC1WSI.dropna(subset=["Phenotype"])
            

In [ ]:
###Adjust wsr8:
####translational adjust
xa=0
ya=0
newX=dfT.X+xa
newY=dfT.Y+ya
x0=0
y0=0
degrees=0

####rotational adjust
def rotate(p, origin=(0, 0), degrees=0):
    angle = np.deg2rad(degrees)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T)

points=list(zip(newX, newY))
origin=(x0,y0)
new_points = rotate(points, origin=origin, degrees=degrees)


###plotting
###tranlational only
plt.figure()
plt.scatter(dfcDC1WSI.X, dfcDC1WSI.Y, color="grey")
plt.scatter(*zip(*points), color="r", alpha=0.02)  
plt.grid()
plt.show()

###tranlational+rotational 
fig=plt.figure()
plt.scatter(dfcDC1WSI.X, dfcDC1WSI.Y, color="grey")
plt.scatter(*zip(*new_points), color="r", alpha=0.02)  
plt.grid()
plt.show()

In [ ]:
# output
dfNewT=dfT[["Sample Name","Phenotype"]]
dfNewT["X"]=list(zip(*new_points))[0]
dfNewT["Y"]=list(zip(*new_points))[1]
dfNewT = dfNewT.round(1)
dfTrimT=dfNewT[(dfNewT.X>0)&(dfNewT.X<20000)&(dfNewT.Y>0)]
#sns.scatterplot(x="X", y="Y", hue="Phenotype", size=0.1,data = dfTrimT)
dfTrimT.to_csv(r"C:\Users\data\output"+ "\\"+dfT["Path"][1]+".txt")
register = pd.DataFrame({"deltaX":xa,"deltaY":ya, "rotation_degree":degrees,"originX":x0, "originY":[y0]})
register.to_csv(r"C:\Users\data\output"+ "\\"+
                "reg_par_"+dfT["Path"][1]+".csv")